In [1]:
import json
import os
from torch.utils.data.dataset import random_split
import csv
from pytorch_transformers import BertTokenizer


In [2]:
tokenizer = BertTokenizer.from_pretrained(
         "../Robera/", do_lower_case=True) 

In [3]:
def list2tokens(strl,tokens,s_index):
    i=0
    t_index=0
    while i<s_index:
        if tokens[t_index]!='[UNK]':
            i+=len(tokens[t_index].strip('#'))
        else:
            i+=1
        t_index+=1
    return t_index


In [12]:
s='公诉机关指控，2013年11月以来，被告人张某某在辽阳市太子河区其租住的平房内多次向栗XX、王XX贩卖毒品。2014年2月在辽纺附近向刘XX贩卖500元冰毒。2014年4月在辽阳市白塔区外环路XX大院门前，张某某向刘XX贩卖500元冰毒。2014年4月，张某某在其租住的平房处向赵XX贩卖麻古15粒。2013年11月，张某某在其租住的平房处向魏XX贩卖500元冰毒。案发后，被告人张某某被公安机关抓获归案。公诉机关认为，被告人张某某的行为已触犯了《中华人民共和国刑法》第三百四十七条之规定，应以贩卖毒品罪追究其刑事责任。'
l=tokenizer.tokenize(s)
st=42
en=st+len("栗XX")
e=list2tokens(s,l,en)
print(s[st:en],e)
l[38:40+1]

栗XX 40


['栗', 'xx', '、']

In [5]:
# 使用bertTokenizer切词
with open('train.json','rt',encoding='utf-8') as fin1,open('test.json','rt',encoding='utf-8') as fin2,\
open('train.sent','w',newline='',encoding='utf-8') as ftrs,open('dev.sent','w',newline='',encoding='utf-8') as fds,\
open('test.sent','w',newline='',encoding='utf-8') as ftes,open('train.pointer','w',newline='',encoding='utf-8') as ftrp,\
open('dev.pointer','w',newline='',encoding='utf-8') as fdp,open('test.pointer','w',newline='',encoding='utf-8') as ftep:

    train_=[]
    for line in fin1:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        train_.append(sentence)
    train_size=len(train_)
    val_size = int(0.25 * len(train_))
    train_,val_= random_split(train_, [train_size-val_size,val_size])
    train_=list(train_)
    val_=list(val_)
    
    for tr in train_:
        sentence_text=tr["sentText"].strip().strip('"')
        ftrs.write(sentence_text+'\n')
        sentence_l=tokenizer.tokenize(sentence_text)
               
        pointer=[]
        for relation in tr['relationMentions']:     
            re_type=relation['label']
            if re_type=='NA':
                continue
            e1s=relation['e1start']
            e1start=list2tokens(sentence_text,sentence_l,e1s)
            e1e=e1s+len(relation['em1Text'])-1
            e1end=list2tokens(sentence_text,sentence_l,e1e)
            e2s=relation['e21start']
            e2start=list2tokens(sentence_text,sentence_l,e2s)
            e2e=e2s+len(relation['em2Text'])-1
            e2end=list2tokens(sentence_text,sentence_l,e2e)
#             print(sentence_l[e1start: e1end + 1])
            if e1end>512 or e2e>512:
                continue
            e1start=str(e1start)
            e1end=str(e1end)
            e2start=str(e2start)
            e2end=str(e2end)
            
            pointer.append(' '.join([e1start,e1end,e2start,e2end,re_type]))
        
        ftrp.write(' | '.join(pointer)+'\n')
           
    for vl in val_:
        sentence_text=vl["sentText"].strip().strip('"')
        fds.write(sentence_text+'\n')
        sentence_l=tokenizer.tokenize(sentence_text)
        
        pointer=[]
        for relation in vl['relationMentions']:
            e1s=relation['e1start']
#             print(e1s,len(sentence_text),len(sentence_l))
            e1start=list2tokens(sentence_text,sentence_l,e1s)
            e1e=e1s+len(relation['em1Text'])-1
            e1end=list2tokens(sentence_text,sentence_l,e1e)
            e2s=relation['e21start']
            e2start=list2tokens(sentence_text,sentence_l,e2s)
            e2e=e2s+len(relation['em2Text'])-1
            e2end=list2tokens(sentence_text,sentence_l,e2e)
            re_type=relation['label']
            if re_type=='NA':
                continue
            if e1end>512 or e2end>512:
                continue
            e1start=str(e1start)
            e1end=str(e1end)
            e2start=str(e2start)
            e2end=str(e2end)
            
            pointer.append(' '.join([e1start,e1end,e2start,e2end,re_type]))
        fdp.write(' | '.join(pointer)+'\n')
        
    test_=[]
    for line in fin2:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        test_.append(sentence)
    
    for te in test_:
        sentence_text=te["sentText"].strip().strip('"')
        ftes.write(sentence_text+'\n')
        sentence_l=tokenizer.tokenize(sentence_text)
        
        pointer=[]
        for relation in te['relationMentions']:
            e1s=relation['e1start']
            e1start=list2tokens(sentence_text,sentence_l,e1s)
            e1e=e1s+len(relation['em1Text'])-1
            e1end=list2tokens(sentence_text,sentence_l,e1e)
            e2s=relation['e21start']
            e2start=list2tokens(sentence_text,sentence_l,e2s)
            e2e=e2s+len(relation['em2Text'])-1
            e2end=list2tokens(sentence_text,sentence_l,e2e)
            re_type=relation['label']
            if re_type=='NA':
                continue
            if e1end>512 or e2end>512:
                continue
            e1start=str(e1start)
            e1end=str(e1end)
            e2start=str(e2start)
            e2end=str(e2end)
            
            pointer.append(' '.join([e1start,e1end,e2start,e2end,re_type]))
        ftep.write(' | '.join(pointer)+'\n')


In [3]:
with open('train.json','rt',encoding='utf-8') as fin1,open('test.json','rt',encoding='utf-8') as fin2,\
open('train.tup','w',newline='',encoding='utf-8') as ftrt,open('dev.tup','w',newline='',encoding='utf-8') as fdt,\
open('test.tup','w',newline='',encoding='utf-8') as ftet:
#     train_=[]
#     for line in fin1:
#         line = line.strip()
#         if not line:
#             continue  #结束则跳出循环
#         sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
#         train_.append(sentence)
#     train_size=len(train_)
#     val_size = int(0.25 * len(train_))
#     train_,val_= random_split(train_, [train_size-val_size,val_size])
#     train_=list(train_)
#     val_=list(val_)
    
    for tr in train_:
        tuples=[]
        for relation in tr['relationMentions']:
            e1=relation['em1Text']
            e2=relation['em2Text']
            re_type=relation['label']
            if re_type=='NA':
                continue
            tuples.append(' ; '.join([e1,e2,re_type]))
        ftrt.write(' | '.join(tuples)+'\n')
           
    for vl in val_:
        
        tuples=[]
        for relation in vl['relationMentions']:
            e1=relation['em1Text']
            e2=relation['em2Text']
            re_type=relation['label']
            if re_type=='NA':
                continue
            tuples.append(' ; '.join([e1,e2,re_type]))
        fdt.write(' | '.join(tuples)+'\n')
        
    test_=[]
    for line in fin2:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        test_.append(sentence)
    
    
    for te in test_:
        tuples=[]
        for relation in te['relationMentions']:
            e1=relation['em1Text']
            e2=relation['em2Text']
            re_type=relation['label']
            if re_type=='NA':
                continue
            tuples.append(' ; '.join([e1,e2,re_type]))
        ftet.write(' | '.join(tuples)+'\n')


In [2]:
# 使用list()切词，以字符为单位
with open('train.json','rt',encoding='utf-8') as fin1,open('test.json','rt',encoding='utf-8') as fin2,\
open('train.sent','w',newline='',encoding='utf-8') as ftrs,open('dev.sent','w',newline='',encoding='utf-8') as fds,\
open('test.sent','w',newline='',encoding='utf-8') as ftes,open('train.pointer','w',newline='',encoding='utf-8') as ftrp,\
open('dev.pointer','w',newline='',encoding='utf-8') as fdp,open('test.pointer','w',newline='',encoding='utf-8') as ftep:

    train_=[]
    for line in fin1:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        train_.append(sentence)
    train_size=len(train_)
    val_size = int(0.25 * len(train_))
    train_,val_= random_split(train_, [train_size-val_size,val_size])
    train_=list(train_)
    val_=list(val_)
    
    for tr in train_:
        sentence_text=tr["sentText"].strip().strip('"')
        ftrs.write(sentence_text+'\n')
        pointer=[]
        for relation in tr['relationMentions']:
            e1start=relation['e1start']
            e1end=e1start+len(relation['em1Text'])-1
            e2start=relation['e21start']
            e2end=e2start+len(relation['em2Text'])-1
            re_type=relation['label']
            if re_type=='NA':
                continue
            if e1end>512 or e2end>512:
                continue
            e1start=str(e1start)
            e1end=str(e1end)
            e2start=str(e2start)
            e2end=str(e2end)
            
            pointer.append(' '.join([e1start,e1end,e2start,e2end,re_type]))
        ftrp.write(' | '.join(pointer)+'\n')
           
    for vl in val_:
        sentence_text=vl["sentText"].strip().strip('"')
        fds.write(sentence_text+'\n')
        pointer=[]
        for relation in vl['relationMentions']:
            e1start=relation['e1start']
            e1end=e1start+len(relation['em1Text'])-1
            e2start=relation['e21start']
            e2end=e2start+len(relation['em2Text'])-1
            re_type=relation['label']
            if re_type=='NA':
                continue
            if e1end>512 or e2end>512:
                continue
            e1start=str(e1start)
            e1end=str(e1end)
            e2start=str(e2start)
            e2end=str(e2end)
            
            pointer.append(' '.join([e1start,e1end,e2start,e2end,re_type]))
        fdp.write(' | '.join(pointer)+'\n')
        
    test_=[]
    for line in fin2:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        test_.append(sentence)
    
    for te in test_:
        sentence_text=te["sentText"].strip().strip('"')
        ftes.write(sentence_text+'\n')
        pointer=[]
        for relation in te['relationMentions']:
            e1start=relation['e1start']
            e1end=e1start+len(relation['em1Text'])-1
            e2start=relation['e21start']
            e2end=e2start+len(relation['em2Text'])-1
            re_type=relation['label']
            if re_type=='NA':
                continue
            if e1end>512 or e2end>512:
                continue
            e1start=str(e1start)
            e1end=str(e1end)
            e2start=str(e2start)
            e2end=str(e2end)
            
            pointer.append(' '.join([e1start,e1end,e2start,e2end,re_type]))
        ftep.write(' | '.join(pointer)+'\n')


In [4]:
' '.join([str(1),str(2),str(3)])

'1 2 3'